Завдання 7. Метод північно-західного кута

In [13]:
import numpy as np

In [7]:
# Задані дані
supply = np.array([230, 250, 170])
demand = np.array([140, 90, 160, 110, 150])

In [8]:
def north_west_corner(supply, demand):
    # Копії масивів початкових запасів та потреб
    supply_copy = supply.copy()
    demand_copy = demand.copy()

    # Індекси поточних рядка та стовпця
    i = 0
    j = 0

    # Список для зберігання кількостей перевезень та їх координат
    bfs = []

    # Поки не визначено кількість перевезень для всіх шляхів
    while len(bfs) < len(supply) + len(demand) - 1:
        # Визначення обсягу перевезення для поточного шляху
        s = supply_copy[i]
        d = demand_copy[j]
        v = min(s, d)

        # Віднімання перевезеного обсягу від запасу та потреби
        supply_copy[i] -= v
        demand_copy[j] -= v

        # Додавання інформації про перевезення у список
        bfs.append(((i, j), v))

        # Перевірка, чи поточний запас вичерпано, та зміна індексу рядка
        if supply_copy[i] == 0 and i < len(supply) - 1:
            i += 1
        # Перевірка, чи поточна потреба вичерпана, та зміна індексу стовпця
        elif demand_copy[j] == 0 and j < len(demand) - 1:
            j += 1

    # Повертаємо список перевезень та їх кількостей
    return bfs


In [9]:
bfs = north_west_corner(supply, demand)
print(bfs)

[((0, 0), 140), ((0, 1), 90), ((1, 1), 0), ((1, 2), 160), ((1, 3), 90), ((2, 3), 20), ((2, 4), 150)]


Завдання 4. Метод потенціалів

In [1]:
from pulp import *

In [2]:
# список всіх вузлів постачання
warehouses = ["A", "B", "C"]

# словник для кількості одиниць постачання для кожного вузла постачання
supply = {"A": 300, "B": 250, "C":200}

# список всіх вузлів отримання
projects = ["1", "2", "3", "4", "5"]

# словник для кількості одиниць вимог для кожного вузла отримання
demand = {
    "1": 210,
    "2": 150,
    "3": 120,
    "4": 135,
    "5": 135,
}

# список вартостей кожного шляху перевезення
costs = [  # Проекти
    [4,8,13,2,7],  # A   Склади
    [9,4,11,9,17],  # B
    [3,16,10,1,4]   # C
]

# Дані про вартість перетворюються в словник
costs = makeDict([warehouses, projects], costs, 0)

In [3]:
# об'єкт 'prob' для збереження даних задачі
prob = LpProblem("Material Supply Problem", LpMinimize)

c:\ida\Lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [4]:
# список кортежів, що містять всі можливі маршрути для транспорту
Routes = [(w, b) for w in warehouses for b in projects]

# словник, що містить змінні, на які посилаються (маршрути)
vars = LpVariable.dicts("Route", (warehouses, projects), 0, None, LpInteger)

In [5]:
#  Мінімізація цільової функції - вартість перевезень
prob += (
    lpSum([vars[w][b] * costs[w][b] for (w, b) in Routes]),
    "Sum_of_Transporting_Costs",
)

In [6]:
# Додаються обмеження максимальних обсягів постачання для кожного вузла постачання (складу)
for w in warehouses:
    prob += (
        lpSum([vars[w][b] for b in projects]) <= supply[w],
        "Sum_of_Products_out_of_warehouses_%s" % w,
    )

# Додаються обмеження мінімальних обсягів вимог для кожного вузла вимог (проекту)
for b in projects:
    prob += (
        lpSum([vars[w][b] for w in warehouses]) >= demand[b],
        "Sum_of_Products_into_projects%s" % b,
    )

In [7]:
    # Розв'язання задачі за допомогою PuLP
    prob.solve()

    # Виведення значень оптимізованих змінних
    for v in prob.variables():
        print(v.name, "=", v.varValue)
        
    # Виведення оптимізованого значення цільової функції на екран
    print("Value of Objective Function = ", value(prob.objective))

Route_A_1 = 165.0
Route_A_2 = 0.0
Route_A_3 = 0.0
Route_A_4 = 135.0
Route_A_5 = 0.0
Route_B_1 = 0.0
Route_B_2 = 150.0
Route_B_3 = 100.0
Route_B_4 = 0.0
Route_B_5 = 0.0
Route_C_1 = 45.0
Route_C_2 = 0.0
Route_C_3 = 20.0
Route_C_4 = 0.0
Route_C_5 = 135.0
Value of Objective Function =  3505.0
